In [4]:
import pandas as pd
from langchain_core.documents import Document
from langchain_groq import ChatGroq
from langchain_community.graphs import Neo4jGraph

In [196]:
df = pd.read_csv('df.csv')

In [ ]:
# !pip install --upgrade --quiet  langchain langchain-community langchain-groq neo4j

In [153]:

groq_api_key="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"
llm=ChatGroq(groq_api_key=groq_api_key,model_name="llama-3.1-70b-versatile")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001CDAF691BE0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001CDAF8B7DA0>, model_name='llama-3.1-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [174]:
NEO4J_URI="neo4j+s://7bd0c2c2.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="mEQ7rVSWNbbpjkQDlAjMx5UuEXB0oeSGtItF7ivpJNw"

In [175]:

graph=Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
)

In [176]:
graph

In [2]:
# !pip install --upgrade --quiet langchain_experimental

In [159]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer=LLMGraphTransformer(llm=llm)

In [160]:
i=0
List_of_relations=[]
List_of_nodes=[]
for paragraph in df['processed_paragraphs']:
    documents=[Document(page_content=paragraph)]
    graph_documents=llm_transformer.convert_to_graph_documents(documents)
    i+=1
    print(i)
    List_of_nodes.append(graph_documents[0].nodes)
    List_of_relations.append(graph_documents[0].relationships)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149


BadRequestError: Error code: 400 - {'error': {'message': "Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '{"nodes": [{"id": "Expected Monetary Value", "type": "Concept"}], "relationships": [{"source_node_id": "Expected Monetary Value", "source_node_type": "Concept", "target_node_id": "weighted average", "target_node_type": "Concept", "type": "TYPE_OF"}]}'}}

In [165]:
rest_of_relations=[]
rest_of_nodes=[]
for i in range(149,len(df)):
    paragraph=df['processed_paragraphs'][i]
    documents=[Document(page_content=paragraph)]
    graph_documents=llm_transformer.convert_to_graph_documents(documents)
    print(i)
    rest_of_nodes.append(graph_documents[0].nodes)
    rest_of_relations.append(graph_documents[0].relationships)

149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166


In [166]:
rest_of_nodes

[[Node(id='Expected Monetary Value', type='Concept', properties={}),
  Node(id='Weighted Average', type='Concept', properties={}),
  Node(id='Expected Cost', type='Concept', properties={}),
  Node(id='Bene T', type='Concept', properties={}),
  Node(id='Outcomes', type='Concept', properties={}),
  Node(id='Probabilities', type='Concept', properties={}),
  Node(id='Event', type='Concept', properties={})],
 [Node(id='Monte Carlo Simulation', type='Approach', properties={}),
  Node(id='Project Objective', type='Objective', properties={}),
  Node(id='Project Schedule', type='Schedule', properties={}),
  Node(id='Cost Estimate', type='Estimate', properties={}),
  Node(id='Project Outcomes', type='Outcomes', properties={}),
  Node(id='Contingency Reserves', type='Reserves', properties={}),
  Node(id='Input Values', type='Values', properties={}),
  Node(id='Histogram', type='Histogram', properties={}),
  Node(id='Output', type='Output', properties={}),
  Node(id='Schedule Risk Results', type='

In [162]:
List_of_nodes


[[Node(id='Project Risk Management', type='Concept', properties={}),
  Node(id='Pmbok Guide Fourth Edition', type='Document', properties={}),
  Node(id='Project Management Institute', type='Organization', properties={}),
  Node(id='Project Risk Management Processes', type='Concept', properties={}),
  Node(id='Plan Risk Management', type='Process', properties={}),
  Node(id='Identify Risks', type='Process', properties={}),
  Node(id='Perform Qualitative Risk Analysis', type='Process', properties={}),
  Node(id='Perform Quantitative Risk Analysis', type='Process', properties={}),
  Node(id='Plan Risk Responses', type='Process', properties={}),
  Node(id='Monitor And Control Risks', type='Process', properties={})],
 [Node(id='Project Risk Management', type='Concept', properties={}),
  Node(id='Pmbok Guide Fourth Edition', type='Publication', properties={}),
  Node(id='Project Managers', type='Person', properties={}),
  Node(id='Project Risk', type='Concept', properties={}),
  Node(id='Pro

In [167]:
relations = List_of_relations + rest_of_relations
nodes = List_of_nodes + rest_of_nodes
print(len(nodes))
print(len(relations))

167
167


In [185]:
df['relations'] = relations
df['nodes'] = nodes


In [186]:
df.to_csv('datafreme_with_relations_and_nodes.csv')

In [170]:
relations[:2]

[[Relationship(source=Node(id='Project Risk Management', type='Concept', properties={}), target=Node(id='Pmbok Guide Fourth Edition', type='Document', properties={}), type='DESCRIBED_IN', properties={}),
  Relationship(source=Node(id='Project Risk Management', type='Concept', properties={}), target=Node(id='Project Management Institute', type='Organization', properties={}), type='PUBLISHED_BY', properties={}),
  Relationship(source=Node(id='Project Risk Management', type='Concept', properties={}), target=Node(id='Project Risk Management Processes', type='Concept', properties={}), type='INCLUDES', properties={}),
  Relationship(source=Node(id='Project Risk Management Processes', type='Concept', properties={}), target=Node(id='Plan Risk Management', type='Process', properties={}), type='INCLUDES', properties={}),
  Relationship(source=Node(id='Project Risk Management Processes', type='Concept', properties={}), target=Node(id='Identify Risks', type='Process', properties={}), type='INCLUDE

In [171]:
# Function to modify node IDs
def modify_node_ids(relations_list):
    for relation_sublist in relations_list:  
        for relation in relation_sublist:  
            # Modify source node ID
            if len(relation.source.id.split()) > 2:
                relation.source.id = relation.source.id.replace(' ', '_')
            
            # Modify target node ID
            if len(relation.target.id.split()) > 2:
                relation.target.id = relation.target.id.replace(' ', '_')


modify_node_ids(relations)


In [178]:
relations[:2]

[[Relationship(source=Node(id='Project_Risk_Management', type='Concept', properties={}), target=Node(id='Pmbok_Guide_Fourth_Edition', type='Document', properties={}), type='DESCRIBED_IN', properties={}),
  Relationship(source=Node(id='Project_Risk_Management', type='Concept', properties={}), target=Node(id='Project_Management_Institute', type='Organization', properties={}), type='PUBLISHED_BY', properties={}),
  Relationship(source=Node(id='Project_Risk_Management', type='Concept', properties={}), target=Node(id='Project_Risk_Management_Processes', type='Concept', properties={}), type='INCLUDES', properties={}),
  Relationship(source=Node(id='Project_Risk_Management_Processes', type='Concept', properties={}), target=Node(id='Plan_Risk_Management', type='Process', properties={}), type='INCLUDES', properties={}),
  Relationship(source=Node(id='Project_Risk_Management_Processes', type='Concept', properties={}), target=Node(id='Identify Risks', type='Process', properties={}), type='INCLUDE

In [180]:
# Function to generate and execute Cypher queries from relationships
def generate_and_execute_cypher_from_relations(graph, relations_list):
    for relation_list in relations_list:
        for relation in relation_list:
            source = relation.source  
            target = relation.target 
            rel_type = relation.type 

            source_label = source.type  
            source_id = source.id      
            
            target_label = target.type 
            target_id = target.id      

            # Generate the Cypher query to MERGE nodes and create the relationship
            query = f"""
            MERGE (a:{source_label} {{id: '{source_id}'}})
            MERGE (b:{target_label} {{id: '{target_id}'}})
            MERGE (a)-[:{rel_type}]->(b);
            """
            
            try:
                # Execute the query inside a transaction
                graph.query(query)
            except Exception as e:
                print(f"Error executing query: {e}")


generate_and_execute_cypher_from_relations(graph, relations)


Error executing query: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'management': expected a parameter, '&', ')', ':', 'WHERE', '{' or '|' (line 2, column 30 (offset: 30))
"            MERGE (a:Project management process {id: 'Project_Risk_Management'})"
                              ^}
Error executing query: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'management': expected a parameter, '&', ')', ':', 'WHERE', '{' or '|' (line 2, column 30 (offset: 30))
"            MERGE (a:Project management process {id: 'Project_Risk_Management'})"
                              ^}
Error executing query: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'management': expected a parameter, '&', ')', ':', 'WHERE', '{' or '|' (line 2, column 30 (offset: 30))
"            MERGE (a:Project management process {id: 'Project_Risk_Management'})"
                              ^}
Error executing query: {code: Neo.ClientError.Statement.Synta

Failed to read from defunct connection IPv4Address(('7bd0c2c2.databases.neo4j.io', 7687)) (ResolvedIPv4Address(('34.78.76.49', 7687)))


Error executing query: Failed to read from defunct connection IPv4Address(('7bd0c2c2.databases.neo4j.io', 7687)) (ResolvedIPv4Address(('34.78.76.49', 7687)))
Error executing query: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'element': expected a parameter, '&', ')', ':', 'WHERE', '{' or '|' (line 2, column 30 (offset: 30))
"            MERGE (a:Project element {id: 'Project Schedule'})"
                              ^}
Error executing query: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'element': expected a parameter, '&', ')', ':', 'WHERE', '{' or '|' (line 2, column 30 (offset: 30))
"            MERGE (a:Project element {id: 'Project Budget'})"
                              ^}
Error executing query: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'element': expected a parameter, '&', ')', ':', 'WHERE', '{' or '|' (line 2, column 30 (offset: 30))
"            MERGE (a:Project element {id: 'Project_Cost_Estimate'